In [ ]:
import logging
from django.conf import settings
from django.http import JsonResponse
from django.views.decorators.http import require_http_methods
from rest_framework.decorators import api_view
import google.generativeai as genai
import json
import os
from staticdata.models import Project
from django.core.exceptions import ValidationError

logger = logging.getLogger(__name__)

# Configure Gemini only once at startup
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")

@api_view(['POST'])
def chatbot_response(request):
    try:
        data = json.loads(request.body.decode('utf-8'))
        user_message = data.get('message', '').strip()
        project_id = data.get('currentAnimation', '').strip()
        
        # Validate input
        if not user_message:
            return JsonResponse({"error": "Message cannot be empty"}, status=400)
        if len(user_message) > 500:
            return JsonResponse({"error": "Message too long (max 500 characters)"}, status=400)

        # Get project details
        try:
            project = Project.objects.get(id=project_id)
        except Project.DoesNotExist:
            return JsonResponse({"error": "Animation project not found"}, status=404)
        except ValidationError:
            return JsonResponse({"error": "Invalid project ID format"}, status=400)

        # Enhanced prompt with actual project description
        prompt = f"""You are an expert animation educator assistant. The user is watching an animation with these details:
        
        **Title**: {project.name}
        **Description**: {project.description}
        
        Provide a concise, accurate answer to their question: {user_message}
        - Keep responses under 300 words
        - Use simple language suitable for learners
        - Include emojis where appropriate
        - Format with line breaks (\n) for readability"""
        
        # Generate response
        response = model.generate_content(
            prompt,
            safety_settings={
                'HARASSMENT': 'BLOCK_NONE',
                'HATE_SPEECH': 'BLOCK_NONE',
                'SEXUAL': 'BLOCK_NONE'
            }
        )

        return JsonResponse({
            "response": response.text,
            "suggestions": generate_suggestions(project)
        })

    except json.JSONDecodeError:
        return JsonResponse({"error": "Invalid JSON format"}, status=400)
    except Exception as e:
        logger.error(f"Chatbot error: {str(e)}", exc_info=True)
        return JsonResponse(
            {"error": "Our assistant is busy learning new things. Try again later!"},
            status=500
        )

def generate_suggestions(project):
    """Generate contextual suggestions based on project content"""
    return [
        f"Explain the main concept of {project.name}",
        f"How does {project.name} work?",
        f"What are the key features of this animation?",
        "Show me a real-world application of this concept"
    ]